In [113]:
import sqlite3

conn = sqlite3.connect('TestDB.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - IMPORT_Courses
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSES''')
c.execute('''CREATE TABLE IMPORT_COURSES(
    [term] TEXT,
    [crn] TEXT,
    [catalog_id] TEXT,
    [section] TEXT,
    [credits] TEXT,
    [title] TEXT,
    [meetings] TEXT,
    [timecodes] TEXT,
    [primary_instructor] TEXT,
    [cap] INT,
    [act] INT, 
    [rem] INT,
    [instructor_fname] TEXT,
    [instructor_lname] TEXT,
    [semester] TEXT,
    [year] TEXT)'''
         )
          
# Create table - IMPORT_COURSE_MEETINGS
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS''')
c.execute('''CREATE TABLE IMPORT_COURSE_MEETINGS (
    [crn] TEXT,
    [location] TEXT,
    [day] TEXT,
    [start] TEXT,
    [end] TEXT,
    [term] TEXT, 
    [building] TEXT, 
    [room] TEXT)'''
         )
        
# Create table - IMPORT_COURSE_CATALOG
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_CATALOG''')

c.execute('''CREATE TABLE IMPORT_COURSE_CATALOG (
    [program_code] TEXT,
    [program_name] TEXT,
    [catalog_ID] TEXT,
    [course_title] TEXT,
    [credits] TEXT,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] TEXT,
    [attributes] TEXT,
    [description] TEXT,
    [EffectiveTerm] TEXT)'''
         )
                 
conn.commit()

# Note that the syntax to create new tables should only be used once in the code (unless you dropped the table/s at the end of the code). 
# The [generated_id] column is used to set an auto-increment ID for each record
# When creating a new table, you can add both the field names as well as the field formats (e.g., Text)

conn.close()




In [114]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///TestDB.db")
IMPORT_Courses = pd.read_sql_table('IMPORT_COURSES',engine)
display(IMPORT_Courses.head())
IMPORT_COURSE_CATALOG = pd.read_sql_table('IMPORT_COURSE_CATALOG',engine)
display(IMPORT_COURSE_CATALOG.head())
IMPORT_COURSE_MEETINGS = pd.read_sql_table('IMPORT_COURSE_MEETINGS',engine)
display(IMPORT_COURSE_MEETINGS.head())

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year


,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,EffectiveTerm


,crn,location,day,start,end,term,building,room


In [7]:
%load_ext sql

# initialize a database connection; may have to adjust username and password 

%sql sqlite:///TestDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @TestDB.db'

In [13]:
%%sql
SELECT  *
FROM IMPORT_COURSE_CATALOG

 * sqlite:///TestDB.db
Done.


program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,EffectiveTerm


In [75]:

import pandas as pd
#creating coureses df
##https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']



# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/courses.csv'.format(semester)
# read all the csv files in that directory (assuming they have the extension courses.csv)

    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)

# print out to a new csv file
result.to_csv('courses_all.csv')


In [76]:
result.head(1).T

,0
term,Fall2014
crn,70384
catalog_id,AC 0011
section,C01
credits,3.000
title,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne
cap,0


In [88]:
import json
import pandas as pd
import re

df = pd.read_csv('courses_all.csv' ,header=0)

df[['instructor_fname','instructor_lname']] = df['primary_instructor'].str.split('  ',expand=True,)
df[['semester','year']] = df['term'].str.split('2',expand=True,)
df['year'] = '2'+df['year']

for i in df.columns[1:]:
    print("       ", df[i].name)
    display(df[i].value_counts().to_frame('count'))
    print('--------')

#df['days'] =df['meetings'][0].strip("\n ' '").split(' ')[1].replace(',',"").replace("'","")
#df['times'] =df['meetings'][0].strip("\n ' '").split(' ')[3].replace(',',"").replace("'","")
#df['dates'] =df['meetings'][0].strip("\n ' '").split(' ')[5].replace(',',"").replace("'","")
#df['start'] =df['meetings'][0].strip("\n ' '").split(' ')[5].split("-")[0].replace(',',"").replace("'","")
#df['end'] =df['meetings'][0].strip("\n ' '").split(' ')[5].split("-")[1].replace(',',"").replace("'","")

#df['location_building'] =df['meetings'][0].strip("\n ' '").split(' ')[7].replace(',',"").replace("'","")
#df['location_building_room'] =df['meetings'][0].strip("\n ' '").split(' ')[8].replace("}]","").replace("'","")
#pd.concat( [dw['meetings'].apply(pd.Series)], axis=1)
#df['meetings'][0]
#pd.concat([df.drop('meetings', axis=1), pd.DataFrame(df['meetings'].tolist())], axis=1)
df.head(2).T



        term


,count
Fall2014,1522
Fall2018,1519
Fall2016,1491
Fall2017,1479
Fall2015,1475
Spring2019,1473
Spring2017,1435
Spring2015,1421
Spring2018,1405
Spring2016,1387


--------
        crn


,count
32768,2
34156,2
75160,2
31890,2
75918,2
...,...
77756,1
34745,1
57266,1
77740,1


--------
        catalog_id


...

,count
Fall,7486
Spring,7121
Summer,1224
Winter,105
SpringBreak,1


--------
        year


,count
2018,3265
2015,3237
2017,3224
2016,3216
2014,1522
2019,1473


--------


,0,1
Unnamed: 0,0,1
term,Fall2014,Fall2014
crn,70384,70385
catalog_id,AC 0011,AC 0011
section,C01,C02
credits,3.000,3.000
title,Introduction to Financial Accounting,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...","[{'days': 'TF', 'times': '0930am-1045am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105'],['TF 0930am-1045am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne,Michael P. Coyne


In [112]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,15937.0,7968.000000,4600.759955,0.0,3984.0,7968.0,11952.0,15936.0
crn,15937.0,55172.299178,19308.879968,10893.0,35426.0,57145.0,74541.0,79215.0
cap,15937.0,17.992784,12.671906,0.0,10.0,19.0,25.0,500.0
act,15937.0,15.578591,10.107690,-1.0,7.0,15.0,23.0,90.0
rem,15937.0,2.477317,12.709781,-75.0,0.0,1.0,5.0,999.0


In [108]:
for i in df.columns[1:]:
    print("       ", df[i].name)
    display(df[i].value_counts().to_frame('count'))
    print('--------')

        term


,count
Fall2014,1522
Fall2018,1519
Fall2016,1491
Fall2017,1479
Fall2015,1475
Spring2019,1473
Spring2017,1435
Spring2015,1421
Spring2018,1405
Spring2016,1387


--------
        crn


,count
32768,2
34156,2
75160,2
31890,2
75918,2
...,...
77756,1
34745,1
57266,1
77740,1


--------
        catalog_id


...

--------
        semester


,count
Fall,7486
Spring,7121
Summer,1224
Winter,105
SpringBreak,1


--------
        year


,count
2018,3265
2015,3237
2017,3224
2016,3216
2014,1522
2019,1473


--------


# imput meetings and timecodes columns
 ---

In [90]:
df.loc[df.meetings == '[]']#.count()

,Unnamed: 0,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year
33,33,Fall2014,72710,AC 0391,A,3.000,Internship,[],[],Tara M. Berwick,0,5,-5,Tara M.,Berwick,Fall,2014
34,34,Fall2014,71227,AC 0392,A,3.000,Internship,[],[],Tara M. Berwick,0,3,-3,Tara M.,Berwick,Fall,2014
45,45,Fall2014,73340,AS 0404,01,3.000,American Studies Graduate Project,[],[],Sally E. O'Driscoll,1,1,0,Sally E.,O'Driscoll,Fall,2014
47,47,Fall2014,73293,AS 0499,01,3.000,Independent Study,[],[],Anna M. Lawrence,0,1,-1,Anna M.,Lawrence,Fall,2014
71,71,Fall2014,70004,AE 0291,F,3.000,Business Ethics,[],[],Lisa M. Sticca-Conrod,20,17,3,Lisa M.,Sticca-Conrod,Fall,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15930,15930,Winter2018,11815,PS 0078,OL1,3.000,The Nature of the Universe,[],[],Joachim Kuhn,20,20,0,Joachim,Kuhn,Winter,2018
15931,15931,Winter2018,11814,PO 0133,OL1,3.000,United States Foreign Policy,[],[],Lucrecia Garcia Iommi,20,11,9,Lucrecia,Garcia Iommi,Winter,2018
15932,15932,Winter2018,11839,PY 0131,OL1,3.000,Abnormal Psychology for Non-Majors,[],[],Margaret M. McClure,20,20,0,Margaret M.,McClure,Winter,2018
15933,15933,Winter2018,11817,RS 0101D,OL1,3.000,Exploring Religion: Religion in a Comparative Key,[],[],Erik J. Ranstrom,20,16,4,Erik J.,Ranstrom,Winter,2018


In [81]:
import numpy as np
df.meetings.loc[df.meetings == '[]'] = np.nan

In [101]:
df.meetings.isna().sum()

2491

In [48]:

df.timecodes.loc[df.timecodes == '[]'].count()

2491

In [49]:
df.timecodes.loc[df.timecodes == '[]']= np.nan

In [50]:
df.timecodes.isna().sum()

2491

In [ ]:
# importing the 

In [1]:

import pandas as pd
#creating course_meetings df
##https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']



# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/course_meetings.csv'.format(semester)
# read all the csv files in that directory (assuming they have the extension course_meetings.csv)

    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)

# print out to a new csv file
result.to_csv('course_meetings_all.csv')



In [83]:
import json
import pandas as pd
import re

df = pd.read_csv('course_meetings_all.csv' )

#df[['instructor_fname','instructor_lname']] = df['primary_instructor'].str.split('  ',expand=True,)
#df[['building','room' , 'NAN']] = df['location'].str.split(' ',expand=True,)
#df['year'] = '2'+df['year']
df.head(1).T


#df['building'] = building

#df['room'] = room

,0
Unnamed: 0,0
term,Fall2014
crn,70384
location,DSB 105
day,T
start,2014-09-02T08:00:00
end,2014-09-02T09:15:00


In [70]:
df['location'].str.split(' ',expand=True,)
list(df.location.unique())

df[df.location =='BH BY ARR']


,Unnamed: 0,term,crn,location,day,start,end
76634,76634,Fall2016,78531,BH BY ARR,R,2016-09-08T15:30:00,2016-09-08T17:30:00
76635,76635,Fall2016,78531,BH BY ARR,R,2016-09-15T15:30:00,2016-09-15T17:30:00
76636,76636,Fall2016,78531,BH BY ARR,R,2016-09-22T15:30:00,2016-09-22T17:30:00
76637,76637,Fall2016,78531,BH BY ARR,R,2016-09-29T15:30:00,2016-09-29T17:30:00
76638,76638,Fall2016,78531,BH BY ARR,R,2016-10-06T15:30:00,2016-10-06T17:30:00
76639,76639,Fall2016,78531,BH BY ARR,R,2016-10-13T15:30:00,2016-10-13T17:30:00
76640,76640,Fall2016,78531,BH BY ARR,R,2016-10-20T15:30:00,2016-10-20T17:30:00
76641,76641,Fall2016,78531,BH BY ARR,R,2016-10-27T15:30:00,2016-10-27T17:30:00
76642,76642,Fall2016,78531,BH BY ARR,R,2016-11-03T15:30:00,2016-11-03T17:30:00
76643,76643,Fall2016,78531,BH BY ARR,R,2016-11-10T15:30:00,2016-11-10T17:30:00


In [71]:
dict_1 = {
'DSB 105':['Dolan School of Business' , 'DSB'],
 'DSB 111':['Dolan School of Business' , 'DSB'],
 'DSB 110A':['Dolan School of Business' , 'DSB'],
 'DSB 108':['Dolan School of Business' , 'DSB'],
 'DSB 110B':['Dolan School of Business' , 'DSB'],
 'DSB 104':['Dolan School of Business' , 'DSB'],
 'DSB 112':['Dolan School of Business' , 'DSB'],
 'DSB 109':['Dolan School of Business' , 'DSB'],
 'DSB 115':['Dolan School of Business' , 'DSB'],
 'DSB 2109A':['Dolan School of Business' , 'DSB'],
 'CNS 5':['Canisius Hall' ,'CNS'],
 'CNS 108':['Canisius Hall' ,'CNS'],
 'CNS 106':['Canisius Hall' ,'CNS'],
 'BNW 254':['Bannow Science Center','BNW'],
 'SON 124':['School of Nursing' , 'SON'],
 'CNS 303':['Canisius Hall' ,'CNS'],
 'CNS 305':['Canisius Hall' ,'CNS'],
 'BNW 137':['Bannow Science Center','BNW'],
 'CNS 15':['Canisius Hall' ,'CNS'],
 'CNS 208':['Canisius Hall' ,'CNS'],
 'CNS 103':['Canisius Hall' ,'CNS'],
 'BLM LL105':['Bellarmine Hall' , 'BLM'],
 'DMH 349':['Donnarumma Hall' , 'DMH'],
 'DMH 350':['Donnarumma Hall' , 'DMH'],
 'CNS 204':['Canisius Hall' ,'CNS'],
 'CNS 207':['Canisius Hall' ,'CNS'],
 'BY ARR ':['By Arrangement' ,'BY ARR '],
 'CNS 6':['Canisius Hall' ,'CNS'],
 'CNS 9':['Canisius Hall' ,'CNS'],
 'CNS 1':['Canisius Hall' ,'CNS'],
 'BNW 332':['Bannow Science Center','BNW'],
 'BNW 256':['Bannow Science Center','BNW'],
 'BNW 254A':['Bannow Science Center','BNW'],
 'SON 203':['School of Nursing' , 'SON'],
 'BNW 240':['Bannow Science Center','BNW'],
 'BNW 245':['Bannow Science Center','BNW'],
 'BNW 261':['Bannow Science Center','BNW'],
 'BNW 264':['Bannow Science Center','BNW'],
 'BNW 319':['Bannow Science Center','BNW'],
 'BNW 369':['Bannow Science Center','BNW'],
 'BNW 333':['Bannow Science Center','BNW'],
 'BNW 166':['Bannow Science Center','BNW'],
 'BNW GR22':['Bannow Science Center','BNW'],
 'BNW 238A':['Bannow Science Center','BNW'],
 'BNW 267':['Bannow Science Center','BNW'],
 'BNW 269':['Bannow Science Center','BNW'],
 'NYS 107C':['DiMenna-Nyselius Library' , 'NYS'],
 'MCA 204':['McAuliffe Hall' , 'MCA'],
 'MCA 206':['McAuliffe Hall' , 'MCA'],
 'BNW 361':['Bannow Science Center','BNW'],
 'BNW 341':['Bannow Science Center','BNW'],
 'BNW 335':['Bannow Science Center','BNW'],
 'BNW 131':['Bannow Science Center','BNW'],
 'BNW 127':['Bannow Science Center','BNW'],
 'BNW 362':['Bannow Science Center','BNW'],
 'BNW 340':['Bannow Science Center','BNW'],
 'BNW 339':['Bannow Science Center','BNW'],
 'BNW 343':['Bannow Science Center','BNW'],
 'BNW 363':['Bannow Science Center','BNW'],
 'CNS 10':['Canisius Hall' ,'CNS'],
 'CNS 202':['Canisius Hall' ,'CNS'],
 'DMH 348':['Donnarumma Hall' , 'DMH'],
 'CNS 203':['Canisius Hall' ,'CNS'],
 'DMH 347':['Donnarumma Hall' , 'DMH'],
 'CNS 206':['Canisius Hall' ,'CNS'],
 'MCA 102':['McAuliffe Hall' , 'MCA'],
 'CNS 2':['Canisius Hall' ,'CNS'],
 'CNS 301':['Canisius Hall' ,'CNS'],
 'DMH 231':['Donnarumma Hall' , 'DMH'],
 'BNW 129B':['Bannow Science Center','BNW'],
 'CNS 304':['Canisius Hall' ,'CNS'],
 'CNS 101':['Canisius Hall' ,'CNS'],
 'CNS 306':['Canisius Hall' ,'CNS'],
 'CNS 209':['Canisius Hall' ,'CNS'],
 'LYL 227B':['Loyola Hall', 'LYL'],
 'CNS 104':['Canisius Hall' ,'CNS'],
 'DMH 131':['Donnarumma Hall' , 'DMH'],
 'BNW 469':['Bannow Science Center','BNW'],
 'DMH 149':['Donnarumma Hall' , 'DMH'],
 'DMH 148':['Donnarumma Hall' , 'DMH'],
 'MCA 203':['McAuliffe Hall' , 'MCA'],
 'BNW 133':['Bannow Science Center','BNW'],
 'BNW 139':['Bannow Science Center','BNW'],
 'BNW 253':['Bannow Science Center','BNW'],
 'DLH 107':['Dolan House', 'DLH'],
 'MCA 107':['McAuliffe Hall' , 'MCA'],
 'BNW 334':['Bannow Science Center','BNW'],
 'DMH 331':['Donnarumma Hall' , 'DMH'],
 'XVR 15':['Xavier Hall', 'XVR'],
 'XVR 13':['Xavier Hall', 'XVR'],
 'XVR 5':['Xavier Hall', 'XVR'],
 'DSB 114':['Dolan School of Business' , 'DSB'],
 'DSB 106':['Dolan School of Business' , 'DSB'],
 'FBR ':['Faber Hall' , 'FBR'],
 'EGN ':['Egan Chapel St Ignatius Loyola','EGN'],
 'GNZ AUDI':['Gonzaga Hall', 'GNZ'],
 'CNS 200':['Canisius Hall' ,'CNS'],
 'JGS 25':['Jogues Hall','JGS'],
 'EGN MCGRATH':['Egan Chapel St Ignatius Loyola','EGN'],
 'BCC 200':['Barone Campus Center' , 'BCC'],
 'LYL 2':['Loyola Hall', 'LYL'],
 'DMH 245':['Donnarumma Hall' , 'DMH'],
 'BNW 138':['Bannow Science Center','BNW'],
 'DSB 107':['Dolan School of Business' , 'DSB'],
 'NYS 115':['DiMenna-Nyselius Library' , 'NYS'],
 'SWH 102':['Southwell Hall', 'SWH'],
 'BNW 128':['Bannow Science Center','BNW'],
 'MCA 101':['McAuliffe Hall' , 'MCA'],
 'JGS LAB-A':['Jogues Hall','JGS'],
 'SON 214':['School of Nursing' , 'SON'],
 'JHE ':['Jewish Home for the Elderly Campus | Jewish Senior Services','JHE'],
 'VA ':['VA CT Health Care System Campus | VA CT Health Care System','VA'],
 'LCH ':['Lord Chamberlain Nursing/Rehab Campus | Lord Chamberlain Nursing/Rehab','LCH'],
 'TBA':['To be announced','TBA'],
 'YAL ':['Yale New Haven Hospital Campus | Yale New Haven Hospital','YAL'],
 'STW ':["St Vincent's Westport Campus | St Vincent's Westport",'STW'],
 'SILV ':['Silver Hill Hospital Campus | Silver Hill Hospital','SILV'],
 'GBMH ':['Greater Bpt Mental Health Campus | Greater Bpt Mental Health','GBMH'],
 'SON 217':['School of Nursing' , 'SON'],
 'NOR ':["Norwalk Hospital Campus | Norwalk Hospital","NOR"],
 'STM ':["St Mary's Hospital Campus | St Mary's Hospital",'STM'],
 'GR ':["Greenwich Hospital Campus | Greenwich Hospital","GR"],
 'STAMH ':["Stamford Hospital Campus | Stamford Hospital","STAMH"],
 'DH ':["Danbury Hospital Campus | Danbury Hospital","DH"],
 'STV ':["St Vincent's Hospital Campus | St Vincent's Hospital","STV"],
 'GRF ':["Griffin Hospital Campus | Griffin Hospital","GRF"],
 'SON ANNEX':['School of Nursing' , 'SON'],
 'SON 118':['School of Nursing' , 'SON'],
 'BNW 236':['Bannow Science Center','BNW'],
 'SON 209':['School of Nursing' , 'SON'],
 'BNW 162':['Bannow Science Center','BNW'],
 'BNW 130':['Bannow Science Center','BNW'],
 'BNW 163':['Bannow Science Center','BNW'],
 'LYL 22':['Loyola Hall', 'LYL'],
 'LYL 1':['Loyola Hall', 'LYL'],
 'LYL 8':['Loyola Hall', 'LYL'],
 'LYL 20':['Loyola Hall', 'LYL'],
 'LYL 21':['Loyola Hall', 'LYL'],
 'PTH MAIN':["Fairfield Campus | Pepsico Theatre",'PTH'],
 'NYS 101':['DiMenna-Nyselius Library' , 'NYS'],
 'BNW 124':['Bannow Science Center','BNW'],
 'DW G111':['Dolan Hall West','DW'],
 'DW G101':['Dolan Hall West','DW'],
 'DMH 241':['Donnarumma Hall' , 'DMH'],
 'CMP 313':['Campion Hall','CMP'],
 'CMP 413':['Campion Hall','CMP'],
 'MCK 101':['McAuliffe Hall','MCK'],
 'LYL 145':['Loyola Hall', 'LYL'],
 'DMH 126':['Donnarumma Hall' , 'DMH'],
 'BLM 112':['Bellarmine Hall' , 'BLM'],
 'ONLINE ':['Online', 'ONLINE'],
 'LUDLW ':["Ludlowe Ctr for Health & Rehab Campus | Ludlowe Ctr for Health & Rehab","LUDLW"],
 'HSR ':["Yale New Haven Hospital Saint Raphael Campus | Yale New Haven Hospital Saint Raphael Campus","HSR"],
 'BH ':['Bridgeport Hospital Campus | Bridgeport Hospital', 'BH'],
 'SON 212':['School of Nursing' , 'SON'],
 'MCA 112':['McAuliffe Hall' , 'MCA'],
 'QCK SCENE':["Quick Center for the Arts","QCK SCENE"],
 'BNW 329':['Bannow Science Center','BNW'],
 'DW G108':['Dolan Hall West','DW'],
 'FBR 1DR1':['Faber Hall' , 'FBR'],
 'BNW 136':['Bannow Science Center','BNW'],
 'BW ':["Bishop Wicke Health Center Campus | Bishop Wicke Health Center","BW"],
 'NWG ':["Nathaniel Witherell, Greenwich Campus | Nathaniel Witherell, Greenwich",'NWG'],
 'MCA 202':['McAuliffe Hall' , 'MCA'],
 'NCRGUA ':["Managua, Nicaragua Campus | Nicaragua (Nursing)","NCRGUA"],
 'BH BY ARR':['Bridgeport Hospital | Room By Arrangement' ,'BH BY ARR'],
 'PTH DESIGN':["Fairfield Campus | Pepsico Theatre",'PTH'],
 'NHS 210':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 224':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 411':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 405':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 403':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 203':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 404':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 410':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'EGN COMM':['Egan Chapel St Ignatius Loyola','EGN'],
 'NYS 114':['DiMenna-Nyselius Library' , 'NYS'],
 'NHS 103':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 405A':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 104':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 308':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'WTBRY ':["Waterbury Hospital Campus | Waterbury Hospital","WTBRY"],
 'CMC ':["CT Children's Medical Center Campus | Children's Med Ctr, Hartford",'CMC'],
 'NHS 208':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 204':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 407':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'LNG 109':['42 Langguth Hall', 'LNG'],
 'DW G109':['Dolan Hall West','DW'],
 'NHS 301':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'BNW 144':['Bannow Science Center','BNW'],
 'NHS 403A':['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'DMH 230':['Donnarumma Hall' , 'DMH'],
 'FPS ':["Fairfield Public Schools Campus | Fairfield Public Schools","FPS"],
 'BD ':["Blythedale Hospital Campus | Blythedale Children's Hospital","BD"],
 'NHD ':["Norwalk Health Department Campus | Norwalk Health Department","NHD"],
 'RVNA ':["Ridgefield Visiting Nurse Association Campus | Ridgefield Visiting Nurse Association ","RVNA"],
 'SHC ':["Stratford Home Care Campus | Stratford Home Care","SHC"],
 'SON ':['School of Nursing' , 'SON'],
 'DSB 110':['Dolan School of Business' , 'DSB'],
 'XVR TV':['Xavier Hall', 'XVR'],
 'BNW 143':['Bannow Science Center','BNW'],
 'DW G105':['Dolan Hall West','DW'],
 'BPS ':["Bridgeport Public Schools Campus | Bridgeport Public Schools","BPS"],
 'NHS 205' :['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'NHS 103/104' :['Fairfield Campus | Nursing and Health Studies', 'NHS'],
 'BY ARR X' : ['By Arrangement' ,'BY ARR X'],
 'MILY ': ['Milford YMCA' , 'MILY']}
dict_1

location_dict =pd.DataFrame.from_dict(dict_1 ,orient='index', columns=['BuildingName','BuildingCode'])
location_dict = location_dict.reset_index()

location_dict.rename(columns = {'index':'location'}, inplace = True)
location_dict.to_csv("buildings.csv")

class codes:



      

```python

GRF = Griffin Hospital Campus | Griffin Hospital  
BW =  Bishop Wicke Health Center Campus | Bishop Wicke Health Center
WTBRY = Waterbury Hospital Campus | Waterbury Hospital
EGN COMM = Egan Chapel St Ignatius Loyola | Room COMM
NWG = Nathaniel Witherell, Greenwich Campus | Nathaniel Witherell, Greenwich
CMC = CT Children's Medical Center Campus | Children's Med Ctr, Hartford
RVNA = Ridgefield Visiting Nurse Association Campus | Ridgefield Visiting Nurse Association   
NHD = Norwalk Health Department Campus | Norwalk Health Department
BD = Blythedale Hospital Campus | Blythedale Children's Hospital
FPS = Fairfield Public Schools Campus | Fairfield Public Schools
SHC = Stratford Home Care Campus | Stratford Home Care
BY ARR = By Arrangement
BY ARR X = By Arrangement
BNW = Bannow Science Center
CNS = Canisius Hall
DMH = Donnarumma Hall
SON =  School of Nursing
MCA = McAuliffe Hall
DLH = Dolan House
BLM = Bellarmine Hall
ENDRIS = Mystic Campus | Enders Island
XVR = Xavier Hall
FBR = Faber Hall
EGN = Egan Chapel St Ignatius Loyola
GNZ = Gonzaga Hall
DSB = Dolan School of Business
JGS = Jogues Hall
LYL =  Loyola Hall
NYS =  DiMenna-Nyselius Library
SWH = Southwell Hall
DW = Dolan Hall West 
CMP = Campion Hall 
MCK =  McAuliffe Hall
LNG = 42 Langguth Hall   
JHE =  Jewish Home for the Elderly Campus | Jewish Senior Services (JHE)
VA = VA CT Health Care System Campus | VA CT Health Care System
LCH = Lord Chamberlain Nursing/Rehab Campus | Lord Chamberlain Nursing/Rehab
YAL = Yale New Haven Hospital Campus | Yale New Haven Hospital
STW = St Vincent's Westport Campus | St Vincent's Westport 
STV = St Vincent's Hospital Campus | St Vincent's Hospital
BPS = Bridgeport Public Schools Campus | Bridgeport Public Schools
SILV = Silver Hill Hospital Campus | Silver Hill Hospital
GBMH = Greater Bpt Mental Health Campus | Greater Bpt Mental Health
MILY = Milford YMCA
STM = St Mary's Hospital Campus | St Mary's Hospital
GR = Greenwich Hospital Campus | Greenwich Hospital
STAMH = Stamford Hospital Campus | Stamford Hospital
DH = Danbury Hospital Campus | Danbury Hospital
GRF = Griffin Hospital Campus | Griffin Hospital
NOR = Norwalk Hospital Campus | Norwalk Hospital
BH = Bridgeport Hospital Campus | Bridgeport Hospital
LUDLW = Ludlowe Ctr for Health & Rehab Campus | Ludlowe Ctr for Health & Rehab
HSR = Yale New Haven Hospital Saint Raphael Campus | Yale New Haven Hospital Saint Raphael Campus
NHS = Fairfield Campus | Nursing and Health Studies 
NCRGUA = Managua, Nicaragua Campus | Nicaragua (Nursing)
QCK SCENE = Quick Center for the Arts
ONLINE = ONLINE
TBA = To be announced
PTH = Fairfield Campus | Pepsico Theatre 
PTH MAIN = Fairfield Campus | Pepsico Theatre | Room MAIN
BCC = Barone Campus Center
PTH DESIGN = Fairfield Campus | Pepsico Theatre | Room DESIGN

```

In [82]:
df = pd.read_csv('course_meetings_all.csv' ,header=0)

df_b = pd.read_csv('buildings.csv' ,header=0)
print(df.shape)
df = df.merge(df_b  ,left_on='location', right_on='location')


df.pop('Unnamed: 0_y')
df.pop('Unnamed: 0_x')
print(df.shape)

(284907, 7)
(284907, 8)


In [78]:
df.isna().sum()

term            0
crn             0
location        0
day             0
start           0
end             0
BuildingName    0
BuildingCode    0
dtype: int64

In [86]:
for i in df.columns[1:]:
    print("       ", df[i].name)
    display(df[i].value_counts().to_frame('count'))
    print('--------')

        term


,count
Fall2018,30783
Fall2016,28352
Fall2017,28231
Fall2015,27973
Fall2014,27908
Spring2017,27143
Spring2016,27054
Spring2018,26560
Spring2015,26305
Spring2019,25663


--------
        crn


,count
37142,109
37143,109
32164,94
31976,82
75485,81
...,...
34444,1
77289,1
38376,1
58130,1


--------
        location


...

--------
        start


,count
2015-09-03T14:00:00,86
2015-09-17T14:00:00,86
2015-09-10T14:00:00,86
2015-12-03T14:00:00,86
2015-11-05T14:00:00,86
...,...
2016-04-08T15:16:00,1
2016-03-22T15:00:00,1
2015-11-16T10:50:00,1
2018-11-28T13:30:00,1


--------
        end


,count
2016-11-10T12:15:00,67
2018-11-29T12:15:00,67
2016-12-08T12:15:00,67
2018-09-06T12:15:00,67
2016-10-13T12:15:00,67
...,...
2016-08-02T14:30:00,1
2015-03-16T13:40:00,1
2015-04-02T17:20:00,1
2016-09-23T12:00:00,1


--------


In [17]:
df = pd.read_csv('SourceData/Catalogs/CourseCatalog2017_2018.csv' ,header=0)
df

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
0,AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,NaN,NaN,NaN,NaN,Students undertake an individualized program o...
1,AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,NaN,NaN,NaN,NaN,This seminar examines selected topics concerni...
2,BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,NaN,NaN,NaN,This course examines the broad philosophical a...
3,BU,Business,BU 0220,Environmental Law and Policy,3 Credits,NaN,NaN,NaN,"EVME Environmental Studies Major Elective, EVP...",This course surveys issues arising out of fede...
4,BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,NaN,NaN,NaN,This course examines the components of common ...
...,...,...,...,...,...,...,...,...,...,...
2215,CO,Communication,CO 0562,Continuing Thesis Research,3 Credits,NaN,NaN,NaN,NaN,NaN
2216,CO,Communication,CO 0570,Project Proposal,3 Credits,NaN,NaN,NaN,NaN,This course operates as an independent study e...
2217,CO,Communication,CO 0571,Independent Project,3 Credits,NaN,NaN,NaN,NaN,This course operates as an independent study e...
2218,CO,Communication,CO 0572,Continuing Project,3 Credits,NaN,NaN,NaN,NaN,NaN


In [21]:
for i in df.columns[1:]:
    print("       ", df[i].name)
    display(df[i].value_counts().to_frame('count')[:])
    print('--------')

        program_name


,count
English,99
Communication,96
Biology,95
Psychology,85
History,85
...,...
"Russian, Eastern European, and Central Asian Studies",2
Irish Studies,2
English for Engineers,1
Interdisciplinary,1


--------
        catalog_id


,count
BB 0123,1
TA 0300,1
MD 0490,1
HS 0101,1
EN 0109,1
...,...
SE 0534,1
PH 0264,1
PH 0220,1
HI 0247,1


--------
        course_title


...

--------
        attributes


,count
VPCH Visual and Performing Arts Core: History Course,42
VPC2 Visual and Performing Arts Core: Applied Course,31
WDIV World Diversity,29
"GDSA Graphic Design: Studio Arts, VPC2 Visual and Performing Arts Core: Applied Course",23
CARS Catholic Studies: Religious Studies,21
...,...
"ISIF Italian Studies: Italy-Focused, ISIT Italian Studies: Italian, VPCH Visual and Performing Arts Core: History Course",1
"GDAH Graphic Design: Art History, GMEL German Major or Minor Course, GMEN German Course Taught in English, ISIC Italian Studies: Italy Component, JST Judaic Studies Minor, VPCH Visual and Performing Arts Core: History Course",1
"BSAH Black Studies: Arts and Humanities, BSFC Black Studies Focus Course, E_AF English Literature After 1800, FREN French Course Taught in English, WDIV World Diversity",1
"EDCG Educational Studies Cognate, EDDV Educational Studies Diversity, E_AF English Literature After 1800, UDIV U.S. Diversity",1


--------
        description


,count
"This course, a continuation of",22
Students may take up to two semesters of a department-approved internship. Students must have a GPA of 2.5 or higher.,8
"Students may take two semesters of internship, approved by the department. Students must have a GPA of 2.5 or higher. Enrollment by permission only.",8
"This course requires a research thesis involving laboratory investigation. Seniors and qualified juniors obtain the consent of the professor supervising their research interest area prior to registering for this program. Past topics include aquatic ecology, bacterial ecology and physiology, biochemistry, cell-wall biosynthesis, evolution of marine invertebrates, genetic regulation of animal development, mammalian physiology, plant biostimulants, plant/insect ecology, population and disease dynamics of shellfish, and signal transduction/gene regulations.",6
"This series of courses represents opportunities (up to six semesters) for Biology majors to gain experience in teaching practices in the laboratory as classroom setting. Students will be paired with an instructor, and assist in instruction, grading and overall successful running of a laboratory section. Strong prior performance in the laboratory to which the student will be assigned is required. Selection will be made by the biology department, after a general call is put out to all students who may be interested in the opportunity. Enrollment by permission only.",6
...,...
"Candidates will learn the characteristics of a variety of reading and learning disabilities, including dyslexia, language comprehension and written expression disorders, and how the symptoms may change in response to development and instruction. Theory and practice of literacy interventions, and appropriate commercial instructional materials for individuals experiencing reading difficulties will also be covered with a focus on data-driven interventions to meet the needs of all children, including culturally and linguistically diverse learners. Use of technology to support the remedial reading and remedial language arts program and in reading intervention is included in this course.",1
"This course provides a detailed discussion of basic organic, inorganic and biochemical principles, and basic physical principles as they related to nurse anesthesia practice. Emphasis is placed upon the behavior of gases and the gas laws, principles of diffusion, gas flow and resistance, acid-base balance, and the biochemical processes necessary for basic cellular function. In addition, the basic physical principles of work, energy, light, electricity and radiation are discussed with a focus on operating room safety and the use of biomedical equipment. Group discussion is utilized to facilitate the application of these principles to the clinical setting.",1
"Designed to familiarize the mainstream teacher with the learning needs of children and adolescents who are linguistically and culturally diverse, this course employs an overview of second language acquisition theory as the framework for discussing ways to meet the needs of English language learners. Teachers also learn strategies for developing and adapting materials for creating communicative classroom environments and assessing student performance.",1
"As the first practicum in the nurse-midwifery specialty, this clinical focuses on providing evidence-based care and utilizing health promotion when caring for the pregnant woman. Emphasis is placed on the physical exam of a pregnant woman, assessment of fetal wellbeing, and providing prenatal education. Students use critical thinking and clinical judgment as they relate the provision of care to pregnant woman. Evidence based-practice guidelines are followed to provide routine prenatal care as well as to recognize abnormal findings. The assessment, diagnosis, treatment, management and evaluation of risk factors and health problems will be addressed. Attention is given to cultural, socioeconomic and family va

--------
